# Sistema de Recomendações Personalizado

Este notebook permite criar um perfil de usuário personalizado, avaliar filmes que você já assistiu e obter recomendações personalizadas baseadas no seu gosto.

In [1]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
import time
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
import random  # Adicionando para seleção aleatória de filmes

# Carregar os dados dos arquivos CSV
movies = pd.read_csv('data/movies.csv')
ratings = pd.read_csv('data/ratings.csv')

print(f"Dados carregados com sucesso!")
print(f"Total de {len(movies)} filmes e {len(ratings)} avaliações.")

# Processar os gêneros dos filmes (transformar de string para lista)
movies['genres'] = movies['genres'].apply(lambda x: x.split('|'))

Dados carregados com sucesso!
Total de 9742 filmes e 100836 avaliações.


In [2]:
# Função para criar um novo perfil de usuário - versão otimizada

def create_user_profile(movies_df, existing_ratings_df, new_user_id=None, num_popular_movies=10, max_search_results=5):
    """
    Cria um novo perfil de usuário e permite avaliar filmes para obter recomendações personalizadas.
    Versão otimizada para performance melhorada.
    
    Args:
        movies_df: DataFrame contendo informações dos filmes
        existing_ratings_df: DataFrame contendo avaliações existentes
        new_user_id: ID opcional para o novo usuário. Se None, um será gerado.
        num_popular_movies: Número de filmes populares para mostrar inicialmente
        max_search_results: Número máximo de resultados de busca para mostrar
        
    Returns:
        new_ratings_df: DataFrame com as avaliações do novo usuário adicionadas
        user_id: O ID do novo usuário
    """
    # Se nenhum user_id for fornecido, gere um pegando o max user_id + 1
    if new_user_id is None:
        new_user_id = existing_ratings_df['userId'].max() + 1
    
    print(f"Criando um novo usuário com ID: {new_user_id}")
    
    # Criar uma lista vazia para armazenar as novas avaliações
    new_ratings = []
    
    # Pré-calcular os filmes populares (filmes com mais avaliações)
    print("Carregando filmes populares...")
    popular_movies = existing_ratings_df.groupby('movieId').size().sort_values(ascending=False).head(num_popular_movies)
    popular_movie_ids = popular_movies.index.tolist()
    
    # Alternativamente, selecionar alguns filmes populares aleatoriamente para tornar o processo mais rápido
    popular_movie_ids = random.sample(popular_movie_ids, min(num_popular_movies, len(popular_movie_ids)))
    
    print("\nPor favor, avalie alguns filmes populares (1-5 estrelas, 0 se não assistiu):")
    
    # Caching dos títulos de filmes para evitar consultas repetidas
    movie_titles_cache = {}
    
    for movie_id in popular_movie_ids:
        # Usar o cache para recuperar o título do filme
        if movie_id not in movie_titles_cache:
            movie_row = movies_df[movies_df['movieId'] == movie_id]
            if not movie_row.empty:
                movie_titles_cache[movie_id] = movie_row['title'].values[0]
            else:
                continue  # Pular se não encontrar o filme
        
        movie_title = movie_titles_cache[movie_id]
        valid_rating = False
        
        while not valid_rating:
            try:
                rating_input = input(f"{movie_title}: ")
                
                # Verificar se o usuário digitou 'q' para sair
                if rating_input.lower() == 'q':
                    print("Pulando para a próxima etapa...")
                    break
                    
                rating = float(rating_input)
                if rating == 0:
                    # Pular se o usuário não viu o filme
                    valid_rating = True
                    continue
                elif 0.5 <= rating <= 5:
                    # Adicionar a avaliação à nossa lista
                    new_ratings.append({
                        'userId': new_user_id,
                        'movieId': movie_id,
                        'rating': rating,
                        'timestamp': int(time.time())
                    })
                    valid_rating = True
                else:
                    print("Por favor, digite uma avaliação entre 0.5 e 5, ou 0 se não assistiu.")
            except ValueError:
                print("Por favor, digite um número válido ou 'q' para pular.")
    
    # Permitir que o usuário procure e avalie filmes adicionais com limite de tempo de pesquisa
    search_count = 0
    max_searches = 10  # Limite para o número de pesquisas
    
    print("\nAgora você pode procurar por filmes específicos para avaliar.")
    print("Digite 'pronto' a qualquer momento para finalizar a criação do perfil.")
    print("Digite 'q' a qualquer momento para pular um filme.")
    
    while search_count < max_searches:
        search_term = input("\nProcure por mais filmes para avaliar (ou digite 'pronto' para terminar): ")
        
        if search_term.lower() == 'pronto':
            break
            
        search_count += 1
        
        # Procurar por filmes que contenham o termo de busca (otimizado)
        # Usar concordância exata para melhorar a performance
        matching_movies = movies_df[movies_df['title'].str.contains(search_term, case=False, na=False)].head(max_search_results)
        
        if matching_movies.empty:
            print("Nenhum filme encontrado com esse termo. Tente novamente.")
            continue
        
        # Exibir os resultados limitados
        print("\nFilmes encontrados:")
        for i, (_, movie) in enumerate(matching_movies.iterrows()):
            print(f"{i+1}. {movie['title']}")
        
        # Deixar o usuário selecionar um filme para avaliar
        selection = input("\nDigite o número do filme que deseja avaliar (ou 'p' para procurar novamente): ")
        
        if selection.lower() == 'p':
            continue
            
        try:
            selection_idx = int(selection) - 1
            if 0 <= selection_idx < len(matching_movies):
                selected_movie = matching_movies.iloc[selection_idx]
                movie_id = selected_movie['movieId']
                movie_title = selected_movie['title']
                
                # Verificar se o filme já foi avaliado
                already_rated = any(r['movieId'] == movie_id for r in new_ratings)
                if already_rated:
                    print(f"Você já avaliou {movie_title}. Vamos encontrar outra coisa.")
                    continue
                
                # Obter a avaliação
                valid_rating = False
                while not valid_rating:
                    try:
                        rating_input = input(f"Avaliação para {movie_title} (0.5-5): ")
                        
                        # Verificar se o usuário digitou 'q' para pular
                        if rating_input.lower() == 'q':
                            print("Pulando este filme...")
                            valid_rating = True
                            continue
                            
                        rating = float(rating_input)
                        if 0.5 <= rating <= 5:
                            new_ratings.append({
                                'userId': new_user_id,
                                'movieId': movie_id,
                                'rating': rating,
                                'timestamp': int(time.time())
                            })
                            valid_rating = True
                        else:
                            print("Por favor, digite uma avaliação entre 0.5 e 5.")
                    except ValueError:
                        print("Por favor, digite um número válido ou 'q' para pular.")
        except (ValueError, IndexError):
            # Se a entrada não for um número válido, continue procurando
            print("Opção inválida. Vamos tentar outra pesquisa.")
    
    # Criar um DataFrame a partir das novas avaliações
    if new_ratings:
        new_ratings_df = pd.DataFrame(new_ratings)
        
        # Combinar com as avaliações existentes
        combined_ratings_df = pd.concat([existing_ratings_df, new_ratings_df], ignore_index=True)
        
        print(f"\nObrigado! Você avaliou {len(new_ratings)} filmes.")
        
        return combined_ratings_df, new_user_id
    else:
        print("\nVocê não avaliou nenhum filme. Vamos usar um perfil anônimo para recomendações genéricas.")
        # Retornar as avaliações existentes sem alterações e um ID de usuário que não existe
        # para forçar as recomendações baseadas apenas em médias bayesianas
        return existing_ratings_df, -1  # ID -1 não existirá e forçará recomendações genéricas

In [5]:
# Função para gerar recomendações personalizadas

def get_personal_recommendations(user_id, ratings_df, movies_df, n_recommendations=10):
    """
    Gera recomendações personalizadas para um usuário com base em suas avaliações.
    
    Args:
        user_id: O ID do usuário para gerar recomendações
        ratings_df: DataFrame contendo todas as avaliações
        movies_df: DataFrame contendo informações dos filmes
        n_recommendations: Número de recomendações a retornar
        
    Returns:
        recommendations_df: DataFrame com filmes recomendados
    """
    # Verificar se o usuário existe e tem avaliações
    user_ratings = ratings_df[ratings_df['userId'] == user_id]
    
    # Se o usuário não existe ou não tem avaliações, retornar filmes com as melhores médias bayesianas
    if user_id == -1 or len(user_ratings) == 0:
        print(f"Usuário {user_id} não tem avaliações suficientes. Gerando recomendações com base na popularidade...")
        return get_generic_recommendations(ratings_df, movies_df, n_recommendations)
    
    # Passo 1: Normalizar as avaliações
    def normalize_ratings(df):
        user_mean_ratings = df.groupby('userId')['rating'].mean()
        df_normalized = df.copy()
        df_normalized['rating_normalized'] = df_normalized.apply(
            lambda x: x['rating'] - user_mean_ratings[x['userId']], 
            axis=1
        )
        return df_normalized
    
    ratings_normalized = normalize_ratings(ratings_df)
    
    # Passo 2: Criar matrizes de utilidade
    def create_X(df, rating_column='rating'):
        M = df['userId'].nunique()
        N = df['movieId'].nunique()
        
        user_mapper = dict(zip(np.unique(df["userId"]), list(range(M))))
        movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(N))))
        
        user_inv_mapper = dict(zip(list(range(M)), np.unique(df["userId"])))
        movie_inv_mapper = dict(zip(list(range(N)), np.unique(df["movieId"])))
        
        user_index = [user_mapper[i] for i in df['userId']]
        item_index = [movie_mapper[i] for i in df['movieId']]
        
        X = csr_matrix((df[rating_column], (user_index, item_index)), shape=(M, N))
        
        return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper
    
    X_norm, user_mapper_norm, movie_mapper_norm, user_inv_mapper_norm, movie_inv_mapper_norm = create_X(
        ratings_normalized, rating_column='rating_normalized'
    )
    
    # Passo 3: Calcular médias bayesianas
    C = ratings_df.groupby('movieId')['rating'].count().mean()
    m = ratings_df.groupby('movieId')['rating'].mean().mean()
    
    def bayesian_avg(ratings):
        return (C*m + ratings.sum())/(C + ratings.count())
    
    bayesian_avg_ratings = ratings_df.groupby('movieId')['rating'].agg(bayesian_avg).reset_index()
    bayesian_avg_ratings.columns = ['movieId', 'bayesian_avg']
    
    # Passo 4: Gerar recomendações
    if user_id not in user_mapper_norm:
        print(f"Usuário {user_id} não encontrado no conjunto de dados.")
        return get_generic_recommendations(ratings_df, movies_df, n_recommendations)
    
    # Obter o índice do usuário na matriz
    user_idx = user_mapper_norm[user_id]
    user_ratings = X_norm[user_idx].toarray().flatten()
    
    # Encontrar filmes que o usuário não avaliou
    rated_movies_idx = np.where(user_ratings != 0)[0]
    rated_movies_ids = [movie_inv_mapper_norm[idx] for idx in rated_movies_idx]
    
    all_movie_indices = np.arange(X_norm.shape[1])
    unrated_movie_indices = np.setdiff1d(all_movie_indices, rated_movies_idx)
    
    # Calcular similaridade com outros usuários
    user_similarities = X_norm.dot(X_norm[user_idx].T).toarray().flatten()
    user_similarities[user_idx] = 0  # Remover o usuário da sua própria lista de similaridade
    
    # Obter os usuários mais similares
    k = min(50, len(user_similarities) - 1)  # Garantir que k não exceda o número de usuários disponíveis
    if k <= 0:
        return get_generic_recommendations(ratings_df, movies_df, n_recommendations)
        
    most_similar_users = np.argsort(user_similarities)[-k:]
    
    # Prever avaliações para filmes não avaliados
    predictions = {}
    
    for movie_idx in unrated_movie_indices:
        movie_id = movie_inv_mapper_norm[movie_idx]
        
        # Obter avaliações de usuários similares para este filme
        similar_users_ratings = []
        similar_users_similarities = []
        
        for similar_user_idx in most_similar_users:
            rating = X_norm[similar_user_idx, movie_idx]
            if rating != 0:  # Considerar apenas avaliações não-zero
                similar_users_ratings.append(rating)
                similar_users_similarities.append(user_similarities[similar_user_idx])
        
        if len(similar_users_ratings) == 0:
            continue  # Pular se nenhum usuário similar avaliou este filme
        
        # Calcular avaliação prevista
        predicted_norm_rating = np.average(similar_users_ratings, weights=similar_users_similarities)
        
        # Converter de volta para a escala original
        user_mean = ratings_df[ratings_df['userId'] == user_id]['rating'].mean()
        predicted_rating = predicted_norm_rating + user_mean
        
        # Combinar com média bayesiana
        bayesian_rating = bayesian_avg_ratings[bayesian_avg_ratings['movieId'] == movie_id]['bayesian_avg'].values
        if len(bayesian_rating) > 0:
            # Peso igual para filtragem colaborativa e média bayesiana
            combined_rating = 0.5 * predicted_rating + 0.5 * bayesian_rating[0]
            predictions[movie_id] = combined_rating
    
    # Ordenar por avaliação prevista
    sorted_predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
    recommended_movie_ids = [movie_id for movie_id, _ in sorted_predictions[:n_recommendations]]
    
    # Criar um DataFrame de recomendações
    recommendations = []
    for movie_id in recommended_movie_ids:
        movie_info = movies_df[movies_df['movieId'] == movie_id]
        if movie_info.empty:
            continue
            
        movie_info = movie_info.iloc[0]
        bayesian_rating = bayesian_avg_ratings[bayesian_avg_ratings['movieId'] == movie_id]['bayesian_avg'].values[0]
        predicted_rating = predictions[movie_id]
        
        recommendations.append({
            'movieId': movie_id,
            'title': movie_info['title'],
            'genres': "|".join(movie_info['genres']),
            'predicted_rating': predicted_rating,
            'bayesian_avg': bayesian_rating
        })
    
    recommendations_df = pd.DataFrame(recommendations)
    
    return recommendations_df


def get_generic_recommendations(ratings_df, movies_df, n_recommendations=10):
    """
    Gera recomendações genéricas baseadas nas médias bayesianas quando o usuário não tem avaliações suficientes.
    
    Args:
        ratings_df: DataFrame contendo todas as avaliações
        movies_df: DataFrame contendo informações dos filmes
        n_recommendations: Número de recomendações a retornar
        
    Returns:
        recommendations_df: DataFrame com filmes recomendados
    """
    print("Gerando recomendações populares...")
    
    # Calcular médias bayesianas
    C = ratings_df.groupby('movieId')['rating'].count().mean()
    m = ratings_df.groupby('movieId')['rating'].mean().mean()
    
    def bayesian_avg(ratings):
        return (C*m + ratings.sum())/(C + ratings.count())
    
    # Aplicar filtro de número mínimo de avaliações (30 avaliações)
    min_votes = 30
    qualified_movies = ratings_df.groupby('movieId').filter(lambda x: len(x) >= min_votes)
    
    # Calcular as médias bayesianas apenas para filmes qualificados
    bayesian_avg_ratings = qualified_movies.groupby('movieId')['rating'].agg(bayesian_avg).reset_index()
    bayesian_avg_ratings.columns = ['movieId', 'bayesian_avg']
    
    # Ordenar por avaliação bayesiana
    bayesian_avg_ratings = bayesian_avg_ratings.sort_values('bayesian_avg', ascending=False)
    
    # Obter os IDs dos filmes recomendados
    recommended_movie_ids = bayesian_avg_ratings.head(n_recommendations)['movieId'].tolist()
    
    # Criar um DataFrame de recomendações
    recommendations = []
    for movie_id in recommended_movie_ids:
        movie_info = movies_df[movies_df['movieId'] == movie_id]
        if movie_info.empty:
            continue
            
        movie_info = movie_info.iloc[0]
        bayesian_rating = bayesian_avg_ratings[bayesian_avg_ratings['movieId'] == movie_id]['bayesian_avg'].values[0]
        
        recommendations.append({
            'movieId': movie_id,
            'title': movie_info['title'],
            'genres': "|".join(movie_info['genres']),
            'predicted_rating': bayesian_rating,  # Usar a média bayesiana como previsão
            'bayesian_avg': bayesian_rating
        })
    
    recommendations_df = pd.DataFrame(recommendations)
    
    return recommendations_df

## Como usar este sistema de recomendações

Execute as células abaixo para usar o sistema de recomendações:

1. A primeira célula cria seu perfil de usuário pedindo que você avalie alguns filmes populares
2. A segunda célula gera suas recomendações personalizadas com base nas suas avaliações

In [6]:
# Para usar o sistema de recomendações, execute este código
import time

# Criar um novo perfil de usuário e adicionar avaliações
# Agora com número reduzido de filmes populares (5) e resultados de busca (5) para ser mais rápido
combined_ratings, my_user_id = create_user_profile(movies, ratings, num_popular_movies=5, max_search_results=5)

print("\n\nPerfil criado com sucesso! Agora vamos gerar suas recomendações...")

Criando um novo usuário com ID: 611
Carregando filmes populares...

Por favor, avalie alguns filmes populares (1-5 estrelas, 0 se não assistiu):
Por favor, digite um número válido ou 'q' para pular.
Por favor, digite um número válido ou 'q' para pular.
Por favor, digite um número válido ou 'q' para pular.
Por favor, digite um número válido ou 'q' para pular.
Pulando para a próxima etapa...
Pulando para a próxima etapa...
Pulando para a próxima etapa...
Pulando para a próxima etapa...
Pulando para a próxima etapa...
Pulando para a próxima etapa...
Pulando para a próxima etapa...

Agora você pode procurar por filmes específicos para avaliar.
Digite 'pronto' a qualquer momento para finalizar a criação do perfil.
Digite 'q' a qualquer momento para pular um filme.
Pulando para a próxima etapa...

Agora você pode procurar por filmes específicos para avaliar.
Digite 'pronto' a qualquer momento para finalizar a criação do perfil.
Digite 'q' a qualquer momento para pular um filme.

Filmes encon

In [7]:
# Gerar recomendações para o novo usuário
my_recommendations = get_personal_recommendations(my_user_id, combined_ratings, movies, n_recommendations=15)

# Exibir as recomendações
print("\nSuas Recomendações Personalizadas de Filmes:")
for i, (_, movie) in enumerate(my_recommendations.iterrows()):
    print(f"{i+1}. {movie['title']} - Avaliação prevista: {movie['predicted_rating']:.2f}")
    
# Comparar com o que você já avaliou
print("\nFilmes que você já avaliou:")
my_ratings = combined_ratings[combined_ratings['userId'] == my_user_id]
my_rated_movies = my_ratings.merge(movies[['movieId', 'title']], on='movieId')
my_rated_movies = my_rated_movies.sort_values(by='rating', ascending=False)

for _, row in my_rated_movies.iterrows():
    print(f"- {row['title']} (Sua avaliação: {row['rating']})")


Suas Recomendações Personalizadas de Filmes:
1. Come and See (Idi i smotri) (1985) - Avaliação prevista: 5.34
2. Jetée, La (1962) - Avaliação prevista: 5.30
3. World of Tomorrow (2015) - Avaliação prevista: 5.30
4. It's Such a Beautiful Day (2012) - Avaliação prevista: 5.28
5. Match Factory Girl, The (Tulitikkutehtaan tyttö) (1990) - Avaliação prevista: 5.27
6. Holiday Inn (1942) - Avaliação prevista: 5.20
7. Watership Down (1978) - Avaliação prevista: 5.14
8. Reign Over Me (2007) - Avaliação prevista: 5.12
9. Night at the Opera, A (1935) - Avaliação prevista: 5.09
10. White Christmas (1954) - Avaliação prevista: 5.08
11. Frank (2014) - Avaliação prevista: 5.07
12. Wild Bunch, The (1969) - Avaliação prevista: 5.04
13. True Grit (1969) - Avaliação prevista: 5.03
14. Victory (a.k.a. Escape to Victory) (1981) - Avaliação prevista: 5.02
15. Outlaw Josey Wales, The (1976) - Avaliação prevista: 5.01

Filmes que você já avaliou:
- Toy Story (1995) (Sua avaliação: 5.0)
- Forrest Gump (1994) (

## Explicação do Sistema de Recomendações

Este sistema usa técnicas avançadas de filtragem colaborativa para gerar recomendações personalizadas:

1. **Normalização centrada na média**: Remove o viés pessoal das avaliações, permitindo comparações mais justas entre usuários.

2. **Filtragem colaborativa**: Encontra usuários com gostos semelhantes aos seus e recomenda filmes que eles gostaram mas você ainda não assistiu.

3. **Média bayesiana**: Ajuda a lidar com filmes que têm poucas avaliações, evitando que filmes com poucas avaliações (mas boas) dominem as recomendações.

4. **Sistema combinado**: Une todas essas técnicas para oferecer recomendações mais precisas e personalizadas.

Quanto mais filmes você avaliar, melhores serão as recomendações!

## Dicas de Uso

- Para pular todos os filmes populares iniciais, digite 'q' em qualquer avaliação
- Para procurar filmes específicos, digite o nome ou parte do nome do filme
- Você pode avaliar filmes com notas de 0.5 a 5 (em incrementos de 0.5)
- Para sair da avaliação de filmes, digite 'pronto' na busca
- Para pular um filme específico, digite 'q' quando for solicitado a avaliar